## WELCOME TO THE FINALE - PROJECT 3

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK ❤️❤️❤️



<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To Windows PC people - an important announcement</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [7]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

You may need to install node if you don't have it already.

https://chatgpt.com/share/68103af2-e2dc-8012-b259-bc135a23273b

In [4]:
# A python based MCP server

fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params) as server:
    fetch_tools = await server.list_tools()

for tool in fetch_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")

fetch: Fetches a URL from the internet and optionally extracts its contents as markdown.  Although originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.


In [5]:
# A JavaScript based MCP server

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params) as server:
    file_tools = await server.list_tools()

for tool in file_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")

read_file: Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.
read_multiple_files: Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.
write_file: Create a new file or completely overwrite an existing file with new content. Use with caution as it will overwrite existing files without warning. Handles text content with proper encoding. Only works within allowed directories.
edit_file: Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showin

In [9]:
instructions = """
You browse the internet to accomplish your instructions.
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=fetch_params) as fetch_mcp_server:
    async with MCPServerStdio(params=files_params) as files_mcp_server:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4o-mini",
            mcp_servers=[fetch_mcp_server, files_mcp_server]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            display(Markdown(result.final_output))

It seems I can't save files directly to the specified directory. However, here's the summarized Banoffee Pie recipe in Markdown format:

```markdown
# Banoffee Pie Recipe

## Ingredients
- **For the pastry case:**  
  - Butter  
  - Flour  
  - Egg yolk  
  - Sugar  
  - Cold water  

- **For the filling:**  
  - Caramel  
  - Bananas  
  - Double cream  
  - Dark chocolate (for garnish)  

## Method

### Step 1: Make the Pastry Case  
Put the butter and flour in a food processor and pulse until it resembles fresh breadcrumbs. Add the egg yolk and sugar, then pulse to combine. Gradually add cold water until the dough starts to come together.

### Step 2: Chill the Dough  
Bring the dough together by hand, wrap in cling film, and chill in the fridge for 30 minutes.

### Step 3: Bake the Pastry  
Preheat the oven to 190°C (170°C fan). Grease a 23cm tart tin. Roll out the chilled pastry and line the tin, trimming excess. Line with baking parchment and weigh down with beans. Bake for 15 minutes, remove the paper, brush with egg white, and bake for another 15-20 minutes until golden. Cool completely.

### Step 4: Assemble the Pie  
Spread half of the caramel in the pastry case, layer with sliced bananas, then cover with the remaining caramel. Chill in the fridge.

### Step 5: Add Whipped Cream  
Whip the cream until thick and spoon it over the tart. Garnish with grated dark chocolate before serving.
```

You can copy this text and save it as `banoffee.md` on your own system. Let me know if you need anything else!